## Task 4 — Data Reduction (15 points)

In this task, we reduce the size of the training dataset (originally ~62 MB zipped) while preserving classification performance. The objective is to produce reduced versions targeting 50%, 25%, and 10% of the original size.

Our reduction pipeline includes:
- **Fourier Transform**: Keeps only the top energy frequencies (`compress_fourier`)
- **Piecewise Linear Approximation (PLA)**: Removes redundant values based on a maximum error threshold (`compress_pla`)
- **Custom serialization**: Writes compressed signals to `.bin` files compatible with the original parser

The following parameter combinations were used (obtained mostly via trial and error):
- **50%**: `keep_ratio = 0.03`, `max_error = 15.0`
- **25%**: `keep_ratio = 0.01`, `max_error = 22.0`
- **10%**: `keep_ratio = 0.003`, `max_error = 35.0`

* Assuming the original data resides in the folder called `data` located at the parent directory w.r.t. this notebook, the reduced training data files also will be located under that file.

In [5]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

In [6]:
import numpy as np
from scipy.fftpack import fft, ifft

def compress_fourier(signal: np.ndarray, keep_ratio: float = 0.05) -> np.ndarray:
    """
    the main idea of this function is basically lossy compression of signals
    using FT with thresholding
    - transform signals into freq domain using FFT
    - keep freq components which has highest magnitudes and zero the rest (thresholding)
    - transform signals back into time domain using inverse FFT
    """
    coeffs = fft(signal)    # apply fft to signal
    magnitude = np.abs(coeffs)  
    threshold = np.quantile(magnitude, 1 - keep_ratio)  # keep only *keep ratio* amount of signals (w.r.t. magnitudes)
    coeffs[magnitude < threshold] = 0   # masking (thresholding)
    return np.real(ifft(coeffs)).astype(np.int16)   # back into time domain using IFFT (we only need real part)

def compress_pla(signal: np.ndarray, max_error: float = 10.0) -> np.ndarray:
    """
    apply piecewise linear approximation
    reduce the number of points in the signal by removing small fluctuations
    max_error : maximum allowed difference between consecutive retained values If a new point does not 
    differ from the last one by more than this value, it gets discarded
    """

    # algorithm works similar to sliding window technique
    if len(signal) == 0:
        return signal
    output = [signal[0]]
    last = signal[0]
    for x in signal[1:]:
        if abs(x - last) > max_error:
            output.append(x)
            last = x
    return np.array(output, dtype=np.int16)


In [7]:
import struct
import zipfile
from src.parser import read_zip_binary


def write_reduced_bin(signals: list, out_bin_path: str):
    """
    write a list of reduced signals (as np arrays) into a single binary .bin file
    uses the same structure as the original training data
    """
    with open(out_bin_path, "wb") as f:
        for signal in signals:
            f.write(struct.pack("i", len(signal)))
            f.write(signal.tobytes())

def create_reduced_zip(input_zip: str, output_zip: str, keep_ratio: float = 0.05, max_error: float = 10.0):
    """
    basically the main function that compresses the data w.r.t. above logic and create
    corresponding zip files

    - load signals from the original zip
    - compress each signal using fourier and pla
    - write the reduced signals to a binary (.bin) file,
    - zip the .bin file
    """
    # load original data
    print(f"Reading original data from {input_zip}...")
    print(f"Initial file size: {round(os.path.getsize(input_zip) / (1024 * 1024),1)} MBs.")
    signals = read_zip_binary(input_zip)

    # apply compression
    print("Applying Fourier + PLA compression... (wait)")
    reduced_signals = []
    for signal in signals:
        s1 = compress_fourier(signal, keep_ratio=keep_ratio)
        s2 = compress_pla(s1, max_error=max_error)
        reduced_signals.append(s2)

    # write reduced data to binary (.bin) file
    bin_name = os.path.splitext(os.path.basename(output_zip))[0] + '.bin'
    temp_bin_path = f"./{bin_name}"
    write_reduced_bin(reduced_signals, temp_bin_path)

    # zip the bin file
    print(f"Zipping to {output_zip}...")
    with zipfile.ZipFile(output_zip, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        zf.write(temp_bin_path, arcname=bin_name)

    os.remove(temp_bin_path)
    print(f"Done. Final size: {round(os.path.getsize(output_zip) / (1024 * 1024),1)} MBs. \n")


In [8]:
# input path (original)
input_zip = "../data/X_train.zip"

# target file paths (reduced)
output_zip_10 = "../data/X_train_reduced_10.zip"
output_zip_25 = "../data/X_train_reduced_25.zip"
output_zip_50 = "../data/X_train_reduced_50.zip"

# below is the main part of the compression that uses below logic
# and here we control the compression/reduction strength

# reduce to 10 percent
create_reduced_zip(input_zip, output_zip_10, keep_ratio=0.003, max_error=35.0)

# reduce to 25 percent
create_reduced_zip(input_zip, output_zip_25, keep_ratio=0.01, max_error=22.0)

# reduce to 50 percent
create_reduced_zip(input_zip, output_zip_50, keep_ratio=0.03, max_error=15.0)


Reading original data from ../data/X_train.zip...
Initial file size: 61.2 MBs.
Applying Fourier + PLA compression... (wait)
Zipping to ../data/X_train_reduced_10.zip...
Done. Final size: 6.2 MBs. 

Reading original data from ../data/X_train.zip...
Initial file size: 61.2 MBs.
Applying Fourier + PLA compression... (wait)
Zipping to ../data/X_train_reduced_25.zip...
Done. Final size: 15.9 MBs. 

Reading original data from ../data/X_train.zip...
Initial file size: 61.2 MBs.
Applying Fourier + PLA compression... (wait)
Zipping to ../data/X_train_reduced_50.zip...
Done. Final size: 30.2 MBs. 



In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

# load training labels
y_train = pd.read_csv("../data/y_train.csv", header=None)
y_train.columns = ["y"]

def save_stratified_indices(y_train, prefix="stratified_subset", fractions=[0.1, 0.25, 0.5]):   # prefix ~path name
    """
    idea is basically creating subset of training data (10% 25% 50%)
    instead of creating additional zip folders of training data we
    sample through training labels (y) and save the corresponding indices
    into a .npy file. for further use, we can simply use advanced indexing
    functionality of numpy (just as in 01_data_exploration notebook)
    """
    y_train = np.array(y_train)
    all_indices = np.arange(len(y_train))  # 0 to len-1

    for frac in fractions:
        strat_indices, _ = train_test_split(
            all_indices,
            train_size=frac,
            stratify=y_train,
            random_state=1  # assign random state for reproducability
        )
        filename = f"{prefix}_{int(frac * 100)}.npy"    # indices will reside in .npy files
        np.save(filename, strat_indices)
        print(f"Saved {len(strat_indices)} samples ({frac*100:.0f}%) to {filename}")

save_stratified_indices(y_train, prefix="../data/stratified_subset")

# later on those indices in npy files will be used for performance comparison between
# reduced datasets and stratified subsamples

Saved 617 samples (10%) to ../data/stratified_subset_10.npy
Saved 1544 samples (25%) to ../data/stratified_subset_25.npy
Saved 3089 samples (50%) to ../data/stratified_subset_50.npy
